# 4. 윤곽선  (Contours)

* **4.1 개요** : findContours, drawContours
* **4.2 윤곽선 속성** : Moments, approxPolyDP, Convex Hull, Bounding Rectangle/Circle/Ellipse

## 4.1 개요
* `cv2.findContours()`, `cv2.drawContours()` 함수 이용
* 정확도를 높히기 위해서 Binary Image를 사용. threshold나 canny edge 전처리를 하면 좋음.
* `cv2.findContours()` 함수는 원본 이미지를 직접 수정하기 때문에, 원본 이미지를 보존 하려면 Copy해서 사용해야 함.
* 검은색 배경에서 하얀색 대상을 찾는 것이므로, **대상은 흰색, 배경은 검은색**으로 만들어 줘야 함

### findContours
`cv2.findContours(image, mode, method[, contours[, hierarchy[, offset]]])` → `image, contours, hierarchy`
* **image** : 8-bit single-channel image. binary image.
* **mode** : contours를 찾는 방법
    * `cv2.RETR_EXTERNAL` : contours line중 가장 바같쪽 Line만 찾음.
    * `cv2.RETR_LIST` : 모든 contours line을 찾지만, hierachy 관계를 구성하지 않음.
    * `cv2.RETR_CCOMP` : 모든 contours line을 찾으며, hieracy관계는 2-level로 구성함.
    * `cv2.RETR_TREE` : 모든 contours line을 찾으며, 모든 hieracy관계를 구성함.
* **method* : contours를 찾을 때 사용하는 근사치 방법
    * `cv2.CHAIN_APPROX_NONE` : 모든 contours point를 저장.
    * `cv2.CHAIN_APPROX_SIMPLE` : contours line을 그릴 수 있는 point 만 저장. (ex; 사각형이면 4개 point)
    * `cv2.CHAIN_APPROX_TC89_L1` : contours point를 찾는 algorithm
    * `cv2.CHAIN_APPROX_TC89_KCOS` : contours point를 찾는 algorithm

### drawContours
`cv2.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]])` → `dst`
* **image** : 원본 이미지
* **contours** : contours정보.
* **contourIdx** : contours list type에서 몇번째 contours line을 그릴 것인지. -1 이면 전체
* **color** : contours line color
* **thickness** : contours line의 두께. 음수이면 contours line의 내부를 채움.

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

# 이미지
img = cv2.imread('images/pringles.jpg', cv2.IMREAD_COLOR)
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 전처리 : 흑백이미지로 변환 > 이진화 > 적당히 뭉개기
kernel = np.ones((5,5),np.uint8)
thresh = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
thresh = cv2.adaptiveThreshold(thresh,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 5) # erode(깍기) -> dilate(불리기)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations = 2) # dilate -> erode

# 윤곽선 검출
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# 원본이미지에 초록색으로 윤곽선 그리기
dst = rgb.copy()
dst = cv2.drawContours(dst, contours, -1, (0,255,0), 3)

# 화면출력
plt.figure(figsize=(20,8))
plt.subplot(1,2,1),plt.imshow(rgb)
plt.title('Original'), plt.xticks([]), plt.yticks([])
plt.subplot(1,2,2),plt.imshow(dst)
plt.title('Output'), plt.xticks([]), plt.yticks([])
    
plt.show()

## 4.2 윤곽선 속성

###  Moments
**[Moments](https://en.wikipedia.org/wiki/Image_moment)**는  Area, Perimeter, 중심점 등 대상을 구분할 수 있는 특징을 의미함
* 면적 : ${M}_{00}$
* 중심점 : $\{\bar{x},\bar{y}\} = \{\frac{{M}_{10}}{{M}_{00}},\frac{{M}_{01}}{{M}_{00}}\} $

In [ ]:
# 첫번째 contours의 moment 특징 추출
c = contours[0]
M = cv2.moments(c)

# 윤곽선의 면적
print('area : ', M['m00'])
print('area using contourArea: ', cv2.contourArea(c))

# 중심점
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])
print('centroid : ({},{})'.format(cx, cy))

# 둘레길이 (Contour Perimeter) : 두번째 파라메터가 True면 폐곡선만들어서 둘레구함, False인 경우 선 길이만 구함
print('perimeter : ',cv2.arcLength(c,True))

### 다각형인지 확인 (Check Convexivity)

In [ ]:
c = contours[0]
print('Convexivity : ',cv2.isContourConvex(c))

### 윤곽선 근사
* `cv2.findContours()` 함수에 의해서 찾은 contours line은 각각의 contours point를 가지고 있음. 이 Point를 연결하여 Line을 그림.
* 이때 이 point의 수를 줄여 근사한 line을 그릴 때 사용되는 방법.
* Point의 수를 줄이는데 사용되는 방식 : [Douglas-Peucker algorithm](https://en.wikipedia.org/wiki/Ramer-Douglas-Peucker_algorithm)

`cv2.approxPolyDP(curve, epsilon, closed[, approxCurve])` → `approxCurve`
* **curve** : contours point array
* **epsilon** : original cuve와 근사치의 최대거리. 최대거리가 클 수록 더 먼 곳의 Point까지 고려하기 때문에 Point수가 줄어듬.
* **closed** : 폐곡선 여부

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

# 이미지
img = cv2.imread('images/pringles.jpg', cv2.IMREAD_COLOR)
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 전처리 : 흑백이미지로 변환 > 이진화 > 적당히 뭉개기
kernel = np.ones((5,5),np.uint8)
thresh = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(thresh,200,255,cv2.THRESH_BINARY_INV)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 4)

# 윤곽선 검출
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# 적용하는 숫자가 커질 수록 Point의 갯수는 감소
epsilon1 = 0.005*cv2.arcLength(contours[0], True)
epsilon2 = 0.05*cv2.arcLength(contours[0], True)

# 윤곽선 근사
approx1 = cv2.approxPolyDP(contours[0], epsilon1, True)
approx2 = cv2.approxPolyDP(contours[0], epsilon2, True)

print("countours : ",len(contours[0]))
print("0.5% approxy : ",len(approx1))
print("5% approxy : ",len(approx2))

# 원본이미지에 초록색으로 윤곽선 그리기
dst = rgb.copy()
dst = cv2.drawContours(dst, contours, -1, (0,255,0), 3)

img1 = rgb.copy()
img2 = rgb.copy()
cv2.drawContours(img1, [approx1], 0,(0,255,0), 3) # 11개의 Point
cv2.drawContours(img2, [approx2], 0,(0,255,0), 3) # 4개의 Point

# 화면출력
plt.figure(figsize=(16,14))
titles = ['Original', 'Output', '0.5%', '5%']
images = [rgb, dst, img1, img2]
for i in range(4):
    plt.subplot(2,2,i+1), plt.title(titles[i]), plt.imshow(images[i])
    plt.xticks([]), plt.yticks([])
plt.show()

### Convex Hull
* ontours point를 모두 포함하는 볼록한 외관선을 의미.
* 아래 그림에서 붉은 선이 Convex Hull을 나타내고 화살표의 차이가 convexity defect라고 함
* convexity defect는 contours와 hull과의 최대차이를 뜻함
![convexitydefects](https://docs.opencv.org/4.0.1/convexitydefects.jpg)

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

# 이미지
img = cv2.imread('images/star.png', cv2.IMREAD_COLOR)
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 전처리 : 흑백이미지로 변환 > 이진화 > 적당히 뭉개기
kernel = np.ones((5,5),np.uint8)
thresh = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 윤곽선 검출
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cnt = contours[0]

# 원본이미지에 초록색으로 윤곽선 그리기
dst = rgb.copy()
dst = cv2.drawContours(dst, [cnt], -1, (0,255,0), 3)

# 바깥쪽 윤곽선 찾아서 convexHull 그리기
hull = cv2.convexHull(cnt)
cvxhull = rgb.copy()
cvxhull = cv2.drawContours(cvxhull, [hull], -1, (0,255,0), 3)

# 화면출력
plt.figure(figsize=(20,8))
plt.subplot(1,2,1),plt.imshow(dst)
plt.title('Countour'), plt.xticks([]), plt.yticks([])
plt.subplot(1,2,2),plt.imshow(cvxhull)
plt.title('Convex Hull'), plt.xticks([]), plt.yticks([])
    
plt.show()

### 외곽 사각형 (Bounding Rectangle)
* **Straight Bounding Rectangle** : 대상의 Rotation은 무시한 사각형 모양
* **Rotated Rectangle** : 대상을 모두 포함하면서, 최소한의 영역을 차지하는 사각형 모양

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

# 이미지
img = cv2.imread('images/driver.jpg', cv2.IMREAD_COLOR)
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 전처리 : 흑백이미지로 변환 > 이진화 > 적당히 뭉개기
kernel = np.ones((5,5),np.uint8)
thresh = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #흑백이미지로
thresh = cv2.medianBlur(thresh, 5) #뭉개서 노이즈 줄이고
thresh = cv2.adaptiveThreshold(thresh,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2) #이진화
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 1) #자잘한 요소 삭제

# 윤곽선 검출
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cnt = contours[0]

# 원본이미지에 초록색으로 윤곽선 그리기
dst = rgb.copy()
dst = cv2.drawContours(dst, cnt, -1, (0,255,0), 3)

# 외곽사각형
x,y,w,h = cv2.boundingRect(cnt)
img_br = rgb.copy()
img_br = cv2.rectangle(img_br,(x,y),(x+w,y+h),(0,255,0),2)

# 외곽사각형 회전한거
rect = cv2.minAreaRect(cnt)
box = cv2.boxPoints(rect)
box = np.int0(box)
img_rotated = rgb.copy()
img_rotated = cv2.drawContours(img_rotated,[box],0,(0,0,255),2)

# 화면출력
plt.figure(figsize=(10,10))
titles = ['Original', 'Output', 'Bounding Rect', 'Rotated Rect']
images = [rgb, dst, img_br, img_rotated]
for i in range(4):
    plt.subplot(2,2,i+1), plt.title(titles[i]), plt.imshow(images[i])
    plt.xticks([]), plt.yticks([])
plt.show()

### 원과 타원
* `cv.minEnclosingCircle(cnt)` : Contours line을 완전히 포함하는 원 중 가장 작은 원
* `cv.fitEllipse(cnt)` : Contours Line을 둘러싸는 타원

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

# 이미지
img = cv2.imread('images/driver.jpg', cv2.IMREAD_COLOR)
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 전처리 : 흑백이미지로 변환 > 이진화 > 적당히 뭉개기
kernel = np.ones((5,5),np.uint8)
thresh = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #흑백이미지로
thresh = cv2.medianBlur(thresh, 5) #뭉개서 노이즈 줄이고
thresh = cv2.adaptiveThreshold(thresh,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2) #이진화
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 1) #자잘한 요소 삭제

# 윤곽선 검출
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cnt = contours[0]

# 원
(x,y),radius = cv2.minEnclosingCircle(cnt)
center = (int(x),int(y))
radius = int(radius)
img_circle = rgb.copy()
img_circle = cv2.circle(img_circle,center,radius,(0,255,0),2)

# 타원
ellipse = cv2.fitEllipse(cnt)
img_ellipse = rgb.copy()
img_ellipse = cv2.ellipse(img_ellipse,ellipse,(0,255,0),2)

# 화면출력
plt.figure(figsize=(10,5))
titles = ['Circle', 'Ellipse']
images = [img_circle, img_ellipse]
for i in range(2):
    plt.subplot(1,2,i+1), plt.title(titles[i]), plt.imshow(images[i])
    plt.xticks([]), plt.yticks([])
plt.show()